Урок 2. Инструменты работы и визуализации ч.1

Условие: дана таблица с колонками (id, name, salary, managerId), студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [1]:
import pandas as pd

In [9]:
data = {'id': [1, 2, 3, 4],
        'name': ['Joe', 'Henry', 'Sam', 'Max'],
        'salary': [70, 80, 60, 90],
        'managerid': [3, 4, 'Null', 'Null']}
pandas_df = pd.DataFrame(data)
pandas_df

,id,name,salary,managerid
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


Решение:

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [10]:
spark = SparkSession.builder.appName("EmployeeSalaryFilter").getOrCreate()


In [11]:
# Создаем Spark DataFrame на основе данных задания
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()

+---+-----+------+---------+
| id| name|salary|managerid|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



In [13]:
# Склеиваем сотрудников со своими менеджерами
joined_df = spark_df.alias("e").join(spark_df.alias("m"), col("e.managerid") == col("m.id"))
joined_df.show()

+---+-----+------+---------+---+----+------+---------+
| id| name|salary|managerid| id|name|salary|managerid|
+---+-----+------+---------+---+----+------+---------+
|  1|  Joe|    70|        3|  3| Sam|    60|     Null|
|  2|Henry|    80|        4|  4| Max|    90|     Null|
+---+-----+------+---------+---+----+------+---------+



In [15]:
# Выбираем сотрудников, которые зарабатывают больше своих менеджеров
result_df = joined_df.filter(col("e.salary") > col("m.salary")).select(col("e.name"))
result_df.show()

+----+
|name|
+----+
| Joe|
+----+



In [16]:
spark.stop()